In [1]:
# Install required packages (if not installed)
!pip install chromadb pandas sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 41.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.8/510.8 kB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 46.8 MB/s eta 0:

In [2]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import EarlyStoppingCallback
from sklearn.utils import resample
from sklearn.metrics import accuracy_score, f1_score
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
from transformers import AutoModelForSeq2SeqLM, pipeline

In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.utils import resample
from torch.nn import CrossEntropyLoss
import torch

# Load full data
filepath = '/content/slang_dataset_with_contexts_and_intent.csv'
df = pd.read_csv(filepath)

# =======================
# 🟨 Hybrid balancing (downsample + oversample)
# =======================
min_samples = df['intent'].value_counts().min()   # smallest class
max_samples = df['intent'].value_counts().max()   # largest class

balanced_df = pd.concat([
    # Downsample big classes
    resample(df[df['intent'] == label],
             replace=False,  # without replacement
             n_samples=min(len(df[df['intent'] == label]), min_samples * 3), # cap large ones
             random_state=42)
    for label in df['intent'].unique()
] + [
    # Oversample minority classes
    resample(df[df['intent'] == label],
             replace=True,   # with replacement
             n_samples=max(len(df[df['intent'] == label]), min_samples * 3), # boost small ones
             random_state=42)
    for label in df['intent'].unique()
])

# Shuffle
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df['message'] = balanced_df['message'].fillna('').astype(str)

# Label encoding
unique_intents = sorted(balanced_df['intent'].unique())
label_map = {intent: i for i, intent in enumerate(unique_intents)}
balanced_df['labels'] = balanced_df['intent'].map(label_map)
number_of_labels = len(label_map)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(balanced_df)

# Tokenizer
model_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["message"], truncation=True, padding="max_length")

dataset = dataset.map(tokenize_function, batched=True)

# Train/val split
split_dataset = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = split_dataset['train']
eval_dataset = split_dataset['test']

# =======================
# 🟨 Compute class weights
# =======================
from collections import Counter
label_counts = Counter(balanced_df['labels'])
total_samples = len(balanced_df)
class_weights = {i: total_samples/count for i, count in label_counts.items()}

# Convert to tensor for PyTorch loss
weights_tensor = torch.tensor([class_weights[i] for i in range(number_of_labels)], dtype=torch.float)

# =======================
# 🟨 Custom loss with class weights
# =======================
from transformers import Trainer

class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")
        loss_fct = CrossEntropyLoss(weight=weights_tensor.to(logits.device))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=number_of_labels
)

# Metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    # Get all possible labels from the label_map
    all_labels = list(label_map.values())
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted", labels=all_labels, zero_division=0),
        "precision": precision_score(labels, preds, average="weighted", labels=all_labels, zero_division=0),
        "recall": recall_score(labels, preds, average="weighted", labels=all_labels, zero_division=0)
    }

# Training args
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.1,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    fp16=True
)

# Weighted Trainer
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# Train
trainer.train()

Map:   0%|          | 0/30444 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.721000,0.668194,0.934308,0.931245,0.931556,0.934308
2,0.336900,0.428344,0.956972,0.955636,0.956915,0.956972
3,0.161000,0.387382,0.964855,0.963957,0.964068,0.964855


TrainOutput(global_step=18267, training_loss=0.6103544660720579, metrics={'train_runtime': 1853.6256, 'train_samples_per_second': 39.417, 'train_steps_per_second': 9.855, 'total_flos': 9681492185487360.0, 'train_loss': 0.6103544660720579, 'epoch': 3.0})

In [10]:
eval_results = trainer.evaluate()
display(eval_results)

{'eval_loss': 0.3873823583126068,
 'eval_accuracy': 0.9648546559369354,
 'eval_f1': 0.9639569464419644,
 'eval_precision': 0.964067913939778,
 'eval_recall': 0.9648546559369354,
 'eval_runtime': 41.6558,
 'eval_samples_per_second': 146.174,
 'eval_steps_per_second': 36.562,
 'epoch': 3.0}

In [11]:
trainer.save_model("./shona_chatbot_model")
tokenizer.save_pretrained("./shona_chatbot_model")

('./shona_chatbot_model/tokenizer_config.json',
 './shona_chatbot_model/special_tokens_map.json',
 './shona_chatbot_model/vocab.txt',
 './shona_chatbot_model/added_tokens.json',
 './shona_chatbot_model/tokenizer.json')

In [14]:
# Save model and tokenizer
save_dir = "./shona_intent_model"

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

# Also save label map for decoding intents
import json
with open(f"{save_dir}/label_map.json", "w") as f:
    json.dump(label_map, f)


In [17]:
import shutil
from google.colab import files

# Zip the directory
shutil.make_archive("shona_intent_model", 'zip', "shona_intent_model")

# Download the zip file
files.download("shona_intent_model.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import zipfile
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
import json

# Define the path to the zipped model and the directory to extract it
zip_file_path = "shona_intent_model.zip"
extract_dir = "shona_intent_model_extracted"

# Unzip the model
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Load the tokenizer and model
loaded_tokenizer = AutoTokenizer.from_pretrained(extract_dir)
loaded_model = AutoModelForSequenceClassification.from_pretrained(extract_dir)

# Load the label map
with open(f"{extract_dir}/label_map.json", "r") as f:
    loaded_label_map = json.load(f)

# Create a pipeline for inference
loaded_classifier = pipeline(
    "text-classification",
    model=loaded_model,
    tokenizer=loaded_tokenizer,
    return_all_scores=True
)

# Create a mapping from id to label name
id2label = {i: intent for intent, i in loaded_label_map.items()}

# Example usage
text = "Makadii henyu?"
result = loaded_classifier(text)
pred = max(result[0], key=lambda x: x["score"])
label_index = int(pred["label"].replace("LABEL_", ""))
predicted_intent = id2label[label_index]

print(f"Input text: {text}")
print(f"Predicted intent: {predicted_intent} (confidence: {pred['score']:.2f})")

In [16]:
import zipfile
import os
import shutil

# Define the path to the model directory and the zip file
model_dir = "shona_intent_model"
zip_file_path = "shona_intent_model.zip"
extract_dir = "shona_intent_model_extracted_retest" # Use a new directory for the retest

# Re-zip the directory (in case the previous zip was corrupted)
shutil.make_archive(model_dir, 'zip', model_dir)

# Attempt to unzip the file within Colab
try:
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print(f"Successfully unzipped the model to '{extract_dir}' within Colab.")
except zipfile.BadZipFile:
    print(f"Error: The file '{zip_file_path}' is still not a valid zip file in Colab.")
except FileNotFoundError:
    print(f"Error: The file '{zip_file_path}' was not found in Colab. Please ensure the model was saved correctly.")
except Exception as e:
    print(f"An unexpected error occurred during unzipping: {e}")

# Optional: You can add code here to try loading the model from extract_dir_retest
# if the unzipping was successful to further verify the file integrity.

Successfully unzipped the model to 'shona_intent_model_extracted_retest' within Colab.


In [13]:
# === Imports ===
import pandas as pd
import chromadb
from sentence_transformers import SentenceTransformer
from chromadb.config import Settings
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForSequenceClassification

# ===============================
# 1. SETUP DATA & BASELINE MODELS
# ===============================

# Load CSV Data for RAG
filepath = '/content/pace_graduate_programs (1).csv'  # Replace with your file path
df = pd.read_csv(filepath)
program_names = df['Program Name'].tolist()
program_links = df['Program Link'].tolist()

# Embed Program Names
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedding_model.encode(program_names)

# Initialize Chroma Vector Store
chroma_client = chromadb.Client(Settings(anonymized_telemetry=False))
collection = chroma_client.get_or_create_collection(name="pace_programs")

# Add data if collection is empty
if collection.count() == 0:
    collection.add(
        documents=program_names,
        metadatas=[{"link": link} for link in program_links],
        embeddings=embeddings.tolist(),
        ids=[f"id{i}" for i in range(len(program_names))]
    )

# Semantic Search
def search_programs(user_query, top_k=5):
    query_embed = embedding_model.encode([user_query])[0].tolist()
    results = collection.query(query_embeddings=[query_embed], n_results=top_k)
    programs = results['documents'][0]
    metadata = results['metadatas'][0]
    return list(zip(programs, [meta['link'] for meta in metadata]))

# Baseline Flan-T5 Model
baseline_model_name = "google/flan-t5-small"
baseline_tokenizer = AutoTokenizer.from_pretrained(baseline_model_name)
baseline_model = AutoModelForSeq2SeqLM.from_pretrained(baseline_model_name)
baseline_pipeline = pipeline("text2text-generation", model=baseline_model, tokenizer=baseline_tokenizer)

# ===============================
# 2. SETUP SHONA SLANG CLASSIFIER
# ===============================

shona_model_path = "/content/shona_chatbot_model"  # Your fine-tuned model path
shona_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")
shona_model = AutoModelForSequenceClassification.from_pretrained(shona_model_path)
shona_classifier = pipeline("text-classification", model=shona_model, tokenizer=shona_tokenizer, return_all_scores=True)

# Load label map
unique_intents = sorted(pd.read_csv('/content/slang_dataset_with_contexts_and_intent.csv')['intent'].unique())
label_map = {intent: i for i, intent in enumerate(unique_intents)}
id2label = {v: k for k, v in label_map.items()}

# ===============================
# 3. CLASSIFICATION FUNCTIONS
# ===============================

def classify_intent(text):
    result = shona_classifier(text)
    pred = max(result[0], key=lambda x: x["score"])
    label_index = int(pred["label"].replace("LABEL_", ""))
    intent_name = id2label[label_index]
    return intent_name, pred["score"]

# ===============================
# 4. RULE-BASED HANDLER FOR SHONA BOT
# ===============================
def handle_intent_response(intent, user_input):
    intent = intent.lower()
    user_input_lower = user_input.lower()

    # -------------------
    # GREETINGS
    # -------------------
    if intent == "greeting":
        greetings = [
            "Hesi shamwari! Uri sei hako?",
            "Mhoroi, zvakanaka here?",
            "Mangwanani akanaka!",
            "Masikati akanaka, shamwari."
        ]
        return greetings[0]  # could randomize

    elif intent == "religious_greeting":
        return "Mangwanani akanaka hama dzaMwari 🙏🏽. Mwari ngavakuropafadzei."

    elif intent == "farewell":
        return "Sarai zvakanaka! Tichataurirana zvakare."

    # -------------------
    # APPRECIATION / GRATITUDE
    # -------------------
    elif intent == "gratitude":
        return "Ndokutendai zvikuru! Makaita basa."

    elif intent == "celebration":
        return "Makorokoto! Mwari akuropafadzei pamufaro uyu 🎉"

    # -------------------
    # REQUESTS / HELP
    # -------------------
    elif intent == "request":
        return "Ndiri kunzwa chikumbiro chako. Chii chaunoda kubatsirwa nacho?"

    # -------------------
    # RELIGION
    # -------------------
    elif intent == "prayer_or_blessing":
        return "Tinosimudzira minamato yedu kuna Mwari. Garai makasimba 🙏🏽"

    elif intent == "religion":
        return "Bata Department reReligion pa: www.religion@pace.edu"

    # -------------------
    # FAMILY / FRIENDSHIP
    # -------------------
    elif intent == "family":
        return "Mhuri inokosha! pa Pace University tinoda uzive kut you are part of the bigger family?"

    elif intent == "friendship":
        return "Zvirikufamba sei shamwari yangu?"

    elif intent == "romantic":
        return "Aaaah, zviri pachena kuti rudo ruripo ❤️"

    # -------------------
    # EDUCATION / WORK
    # -------------------
    elif intent == "educational":
        return "Zvekudzidza hazviperere pano. Unoda ruzivo pamusoro pemapurogiramu api?"

    elif intent == "work":
        return "bata carear department pa www.carears@pace.edu"

    # -------------------
    # APPLICATION FLOW
    # -------------------
    elif any(word in user_input_lower for word in ["apply", "nyora", "kunyora", "register", "application","program"]):
        name = input("ShonaBot: Ndokumbirawo zita rako rizere: ")
        education = input("ShonaBot: Wakadzidza kupi kare? (e.g., BSc, diploma, etc.): ")
        email = input("ShonaBot: Email yako ndeipi?: ")
        print(f"\n[INFO] Application received:\nName: {name}\nEducation: {education}\nEmail: {email}")
        return "Waita apply! Chikumbiro chako chakatumirwa kuchikoro. Tichakutumira zvimwe ruzivo 💼"

    # -------------------
    # MISC / FALLBACK
    # -------------------
    elif intent == "sympathy":
        return "Ndinonzwa nemi. Mwari vakupai nyaradzo munguva ino yakaoma."

    elif intent == "health_update":
        return "Ndapota chengeta hutano hwako. Kana zviri serious enda kuchipatara."

    elif intent == "sports":
        return "yes we have a lot of sporting activities pa Pace University, bata sports department pa sports@pace.edu"

    elif intent == "humor":
        return "Wandiseka zve! 😂😂😂😂"

    # Default fallback
    return "Ndanzwisisa zvawataura, asi ndinoda rumwe ruzivo kuti ndikubatsire."



# ===============================
# 5. RAG + FLAN-T5 GENERATION
# ===============================

def generate_answer_with_rag(query):
    intent, confidence = classify_intent(query)
    programs = search_programs(query)
    context = "\n".join([f"{name} – {link}" for name, link in programs])
    prompt = f"""You are a helpful assistant that answers questions about graduate programs at Pace University.
The user's intent is: {intent} (confidence: {confidence:.2f}).

Programs:
{context}

Question: {query}

Answer:"""
    answer = baseline_pipeline(prompt, max_new_tokens=150)[0]['generated_text']
    return intent, confidence, answer

# ===============================
# 6. EXIT CHECK
# ===============================

def is_exit(text):
    return text.strip().lower() in ["exit", "quit", "bye"]

# ===============================
# 7. INTERACTIVE CHAT LOOP
# ===============================

print("💬 Welcome! Type your question in Shona slang or English. (Type 'exit' to quit)\n")

while True:
    user_input = input("Iwe: ")

    if is_exit(user_input):
        print("PaceBot: Zvakanaka, tichaonana zvakare! 🎓")
        break

    # Shona chatbot
    intent, confidence = classify_intent(user_input)
    special_response = handle_intent_response(intent, user_input)

    print(f"\n📌 Detected Intent: {intent} (confidence {confidence:.2f})")

    if special_response is not None:
        print(f"🤖 ShonaBot: {special_response}")
    else:
        print("🤖 ShonaBot: Ndine urombo, handina mhinduro yakajeka pane izvozvo.")

    # Baseline RAG
    intent_rag, conf_rag, rag_answer = generate_answer_with_rag(user_input)
    print(f"🤖 Baseline RAG: {rag_answer}")
    print("-" * 100)


Device set to use cuda:0
Device set to use cuda:0
/usr/local/lib/python3.12/dist-packages/transformers/pipelines/text_classification.py:111: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


💬 Welcome! Type your question in Shona slang or English. (Type 'exit' to quit)

Iwe: wadii

📌 Detected Intent: Greeting (confidence 1.00)
🤖 ShonaBot: Hesi shamwari! Uri sei hako?
🤖 Baseline RAG: Ph.D. – https://www.pace.edu/program/talent-management-mba Higher Education Administration and Student Affairs, MA – https://www.pace.edu/graduate-programs/ma-higher-education-administration-student-affairs/ Public Administration, MPA – https://www.pace.edu/graduate-programs/master-public-administration-mpa/ Homeland Security, MA – https://www.pace.edu/graduate-programs/ma-homeland-security/
----------------------------------------------------------------------------------------------------
Iwe: makasimba here jahman

📌 Detected Intent: Greeting (confidence 1.00)
🤖 ShonaBot: Hesi shamwari! Uri sei hako?
🤖 Baseline RAG: Makasimba here Jahman
----------------------------------------------------------------------------------------------------
Iwe: pace inoita mari ?

📌 Detected Intent: Finance (co

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset



📌 Detected Intent: Religion (confidence 1.00)
🤖 ShonaBot: Bata Department reReligion pa: www.religion@pace.edu
🤖 Baseline RAG: Religion
----------------------------------------------------------------------------------------------------
Iwe: wadii

📌 Detected Intent: Greeting (confidence 1.00)
🤖 ShonaBot: Hesi shamwari! Uri sei hako?
🤖 Baseline RAG: Ph.D. – https://www.pace.edu/program/talent-management-mba Higher Education Administration and Student Affairs, MA – https://www.pace.edu/graduate-programs/ma-higher-education-administration-student-affairs/ Public Administration, MPA – https://www.pace.edu/graduate-programs/master-public-administration-mpa/ Homeland Security, MA – https://www.pace.edu/graduate-programs/ma-homeland-security/
----------------------------------------------------------------------------------------------------
Iwe: bhoo ukudii

📌 Detected Intent: general (confidence 1.00)
🤖 ShonaBot: Ndanzwisisa zvawataura, asi ndinoda rumwe ruzivo kuti ndikubatsire.
🤖 Baseli

In [18]:
%pip install huggingface_hub

In [19]:
from huggingface_hub import notebook_login

notebook_login()

In [25]:
from huggingface_hub import create_repo

repo_name = "shona-intent-classification-model"
create_repo(repo_name, exist_ok=True)

RepoUrl('https://huggingface.co/HappymoreMasoka/shona-intent-classification-model', endpoint='https://huggingface.co', repo_type='model', repo_id='HappymoreMasoka/shona-intent-classification-model')

In [21]:
from huggingface_hub import notebook_login

notebook_login()

In [26]:
from huggingface_hub import create_repo

repo_name = "shona-intent-classification-model"
create_repo(repo_name, exist_ok=True)

RepoUrl('https://huggingface.co/HappymoreMasoka/shona-intent-classification-model', endpoint='https://huggingface.co', repo_type='model', repo_id='HappymoreMasoka/shona-intent-classification-model')

In [27]:
from huggingface_hub import notebook_login, create_repo

notebook_login()

repo_name = "shona-intent-classification-model"
create_repo(repo_name, exist_ok=True)

RepoUrl('https://huggingface.co/HappymoreMasoka/shona-intent-classification-model', endpoint='https://huggingface.co', repo_type='model', repo_id='HappymoreMasoka/shona-intent-classification-model')

In [28]:
from huggingface_hub import notebook_login, create_repo

notebook_login()

repo_name = "shona-intent-classification-model"
create_repo(repo_name, exist_ok=True)

RepoUrl('https://huggingface.co/HappymoreMasoka/shona-intent-classification-model', endpoint='https://huggingface.co', repo_type='model', repo_id='HappymoreMasoka/shona-intent-classification-model')

In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline
from transformers.models.distilbert import DistilBertForSequenceClassification # Import the specific model class
import json
from huggingface_hub import hf_hub_download

# Define your model's repository ID on Hugging Face Hub
# Replace "HappymoreMasoka/shona-intent-classification-model" with your actual repo ID
model_repo_id = "HappymoreMasoka/shona-intent-classification-model"

# Load the tokenizer and model directly from the Hub with trust_remote_code=True
loaded_tokenizer = AutoTokenizer.from_pretrained(model_repo_id, trust_remote_code=True)
# Use the specific model class instead of AutoModel
loaded_model = DistilBertForSequenceClassification.from_pretrained(model_repo_id, trust_remote_code=True)

print("Model and tokenizer loaded successfully from Hugging Face Hub.")

# If you also pushed the label_map.json, you can load it as well
try:
    label_map_path = hf_hub_download(repo_id=model_repo_id, filename="label_map.json")
    with open(label_map_path, "r") as f:
        loaded_label_map = json.load(f)
    # Create a mapping from id to label name
    id2label = {i: intent for intent, i in loaded_label_map.items()}
    print("Label map loaded successfully.")
except Exception as e:
    print(f"Could not load label_map.json from the Hub: {e}")
    # If label_map.json is not available, you might need to recreate id2label
    # based on your understanding of the model's output labels.
    id2label = {i: f"LABEL_{i}" for i in range(loaded_model.config.num_labels)}


# Create a pipeline for inference
loaded_classifier = pipeline(
    "text-classification",
    model=loaded_model,
    tokenizer=loaded_tokenizer,
    return_all_scores=True # Or top_k=None for similar functionality
)

print("Text classification pipeline created.")

# Example usage
text = "Mhoroi, makadii?"
result = loaded_classifier(text)
pred = max(result[0], key=lambda x: x["score"])

# Use id2label if successfully loaded, otherwise use the default LABEL_X format
predicted_intent = id2label.get(int(pred["label"].replace("LABEL_", "")), pred["label"])


print(f"Input text: {text}")
print(f"Predicted intent: {predicted_intent} (confidence: {pred['score']:.2f})")

ValueError: Unrecognized model in HappymoreMasoka/shona-intent-classification-model. Should have a `model_type` key in its config.json, or contain one of the following strings in its name: aimv2, aimv2_vision_model, albert, align, altclip, apertus, arcee, aria, aria_text, audio-spectrogram-transformer, autoformer, aya_vision, bamba, bark, bart, beit, bert, bert-generation, big_bird, bigbird_pegasus, biogpt, bit, bitnet, blenderbot, blenderbot-small, blip, blip-2, blip_2_qformer, bloom, bridgetower, bros, camembert, canine, chameleon, chinese_clip, chinese_clip_vision_model, clap, clip, clip_text_model, clip_vision_model, clipseg, clvp, code_llama, codegen, cohere, cohere2, cohere2_vision, colpali, colqwen2, conditional_detr, convbert, convnext, convnextv2, cpmant, csm, ctrl, cvt, d_fine, dab-detr, dac, data2vec-audio, data2vec-text, data2vec-vision, dbrx, deberta, deberta-v2, decision_transformer, deepseek_v2, deepseek_v3, deepseek_vl, deepseek_vl_hybrid, deformable_detr, deit, depth_a

In [32]:
from huggingface_hub import upload_folder
import os

# Define the local directory where the model is saved
local_dir = "./shona_intent_model"

# Define your model's repository ID on Hugging Face Hub
# Replace "HappymoreMasoka/shona-intent-classification-model" with your actual repo ID
model_repo_id = "HappymoreMasoka/shona-intent-classification-model"

# Push the entire directory to the Hub
# This will upload all files in the local_dir to the specified repository
# Use overwrite=True if you want to replace existing files
upload_folder(
    repo_id=model_repo_id,
    folder_path=local_dir,
    commit_message="Re-uploading model files after resolving potential corruption issues",
    # overwrite=True # Uncomment this line if you want to overwrite existing files
)

print(f"Model files from '{local_dir}' pushed to '{model_repo_id}' on Hugging Face Hub.")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...hona_intent_model/model.safetensors:   0%|          |  558kB /  541MB            

Model files from './shona_intent_model' pushed to 'HappymoreMasoka/shona-intent-classification-model' on Hugging Face Hub.
